# HU Extension                     Assignment 02           E63 Big Data Analytics
## Emmanuel Aboah
### Issued on: Feb 03, 2023                            Due by 12 noon EST, Saturday, Feb 11, 2023

### Problem 1.
Consider attached file titanic.txt. It contains: an ordinal number, price of the ticket paid, name of the person, old fashioned gender classification, and the age of a Titanic passenger. Read directly from that file and create and RDD. Display the schema of that RDD. Display the content of that RDD. Similarly, read directly from file books.json and create a DataFrame. Display the schema of that DataFrame. Display the content of that DataFrame. Transform the RDD object into a DataFrame. Convince us that new object is of the expected type. Display the content of new DataFrame. Transform the original DataFrame into an RDD. Convince yourself and us that the transformed object is an RDD

In [1]:
# Imports

from pyspark import SparkContext, SparkConf
from pyspark.sql import Row, SparkSession, SQLContext
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [2]:
# set spark conf and context

conf = SparkConf().setMaster("local").setAppName("Hw01App")

sc = SparkContext(conf=conf)

sc.setLogLevel("ERROR")

your 131072x1 screen size is bogus. expect trouble


23/02/10 19:00:31 WARN Utils: Your hostname, LAPTOP-85L1BUVJ resolves to a loopback address: 127.0.1.1; using 172.22.11.167 instead (on interface eth0)
23/02/10 19:00:31 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/10 19:00:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# SparkSession

spark = SparkSession(sc)

In [4]:
# Read textfile

titanicRdd = sc.textFile("titanic.txt")

In [5]:
# Transform RDD to impose schema

titanicFields = titanicRdd.map(lambda t: t.split(","))

passengers = titanicFields.map(
    lambda p: Row(
        ord_number = p[0], 
        price = p[1], 
        name = p[2], 
        gender = p[3],
        age = p[4]
    )
)

passengers.collect()

[Row(ord_number='1', price='125.00', name='Mr. Owen Harris', gender='male', age='22'),
 Row(ord_number='2', price='75.00', name='Mrs. Florence Briggs Thayer', gender='female', age='38'),
 Row(ord_number='3', price='75.00', name='Madame. Laina Heikkinen', gender='female', age='66'),
 Row(ord_number='4', price='75.00', name='Jacques Heath (Lily May Peel)', gender='female', age='35'),
 Row(ord_number='5', price='125.00', name='William Henry', gender='male', age='35'),
 Row(ord_number='6', price='125.00', name='James Moran', gender='male', age='0'),
 Row(ord_number='7', price='175.00', name='Timothy J McCarthy', gender='male', age='55'),
 Row(ord_number='8', price='25.00', name='Master Gosta Leonard Palsson', gender='male', age='2'),
 Row(ord_number='9', price='75.00', name='Mrs. Oscar Johnson (Elisabeth Vilhelmina Berg)', gender='female', age='27'),
 Row(ord_number='10', price='50.00', name='Miss. Adele Achem', gender='female', age='14')]

In [6]:
# Convert rdd to DF and print schema and show data

passengersDF = passengers.toDF()

passengersDF.printSchema()

passengersDF.show()

root
 |-- ord_number: string (nullable = true)
 |-- price: string (nullable = true)
 |-- name: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- age: string (nullable = true)



+----------+------+--------------------+------+---+
|ord_number| price|                name|gender|age|
+----------+------+--------------------+------+---+
|         1|125.00|     Mr. Owen Harris|  male| 22|
|         2| 75.00|Mrs. Florence Bri...|female| 38|
|         3| 75.00|Madame. Laina Hei...|female| 66|
|         4| 75.00|Jacques Heath (Li...|female| 35|
|         5|125.00|       William Henry|  male| 35|
|         6|125.00|         James Moran|  male|  0|
|         7|175.00|  Timothy J McCarthy|  male| 55|
|         8| 25.00|Master Gosta Leon...|  male|  2|
|         9| 75.00|Mrs. Oscar Johnso...|female| 27|
|        10| 50.00|   Miss. Adele Achem|female| 14|
+----------+------+--------------------+------+---+



In [7]:
# Read from JSON

book_schema = StructType ([
    StructField("title", StringType(), True),
    StructField("author", StringType(), True),
    StructField("published", DateType(), True),
    StructField("publisher", StringType(), True),
    StructField("website", StringType(), True)
])

book_json_df = spark.read.option("multiline", "true").schema(book_schema).json("books.json")


In [8]:
# Display passengersJsonDF schema

book_json_df.printSchema()

root
 |-- title: string (nullable = true)
 |-- author: string (nullable = true)
 |-- published: date (nullable = true)
 |-- publisher: string (nullable = true)
 |-- website: string (nullable = true)



In [9]:
# Show Data

book_json_df.show()

+--------------------+----------------+----------+---------------+--------------------+
|               title|          author| published|      publisher|             website|
+--------------------+----------------+----------+---------------+--------------------+
|Eloquent JavaScri...|Marijn Haverbeke|2014-12-14|No Starch Press|http://eloquentja...|
|Learning JavaScri...|     Addy Osmani|2012-07-01| O'Reilly Media|http://www.addyos...|
+--------------------+----------------+----------+---------------+--------------------+



In [10]:
# Show type

type(book_json_df)

pyspark.sql.dataframe.DataFrame

In [11]:
# Convert to RDD and show type

book_json_df_rdd = book_json_df.rdd

type(book_json_df_rdd)

pyspark.rdd.RDD

## Problem 2.

Read the inferred schema of the DataFrame created when reading file titanic.txt. Modify that schema so that the age of the person is not an Integer or a Long but rather a Float. Try loading titanic.txt with an enforced schema. Report. Make sure that you can load the data with an enforced schema. Display the data content of your DataFrame.

In [12]:
 # Show inferred schema

passengersDF.printSchema()

root
 |-- ord_number: string (nullable = true)
 |-- price: string (nullable = true)
 |-- name: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- age: string (nullable = true)



In [13]:
# Convert afe column to float

converted_col_df = passengersDF.withColumn("age", passengersDF["age"].cast(FloatType()))

converted_col_df.printSchema()

converted_col_df.show()

root
 |-- ord_number: string (nullable = true)
 |-- price: string (nullable = true)
 |-- name: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- age: float (nullable = true)

+----------+------+--------------------+------+----+
|ord_number| price|                name|gender| age|
+----------+------+--------------------+------+----+
|         1|125.00|     Mr. Owen Harris|  male|22.0|
|         2| 75.00|Mrs. Florence Bri...|female|38.0|
|         3| 75.00|Madame. Laina Hei...|female|66.0|
|         4| 75.00|Jacques Heath (Li...|female|35.0|
|         5|125.00|       William Henry|  male|35.0|
|         6|125.00|         James Moran|  male| 0.0|
|         7|175.00|  Timothy J McCarthy|  male|55.0|
|         8| 25.00|Master Gosta Leon...|  male| 2.0|
|         9| 75.00|Mrs. Oscar Johnso...|female|27.0|
|        10| 50.00|   Miss. Adele Achem|female|14.0|
+----------+------+--------------------+------+----+



In [14]:
# Read titanic with enforced schema

passenger_schema = StructType ([
    StructField("ord_number", IntegerType(), True),
    StructField("price", FloatType(), True),
    StructField("name", StringType(), True),
    StructField("gender", StringType(), True),
    StructField("age", FloatType(), True)
])

inf_schema_df = (
    spark.read
    .option("header", False)
    .option("infer_schema", True)
    .schema(passenger_schema)
    .csv("titanic.txt")
)

inf_schema_df.printSchema()

inf_schema_df.show()

root
 |-- ord_number: integer (nullable = true)
 |-- price: float (nullable = true)
 |-- name: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- age: float (nullable = true)

+----------+-----+--------------------+------+----+
|ord_number|price|                name|gender| age|
+----------+-----+--------------------+------+----+
|         1|125.0|     Mr. Owen Harris|  male|22.0|
|         2| 75.0|Mrs. Florence Bri...|female|38.0|
|         3| 75.0|Madame. Laina Hei...|female|66.0|
|         4| 75.0|Jacques Heath (Li...|female|35.0|
|         5|125.0|       William Henry|  male|35.0|
|         6|125.0|         James Moran|  male| 0.0|
|         7|175.0|  Timothy J McCarthy|  male|55.0|
|         8| 25.0|Master Gosta Leon...|  male| 2.0|
|         9| 75.0|Mrs. Oscar Johnso...|female|27.0|
|        10| 50.0|   Miss. Adele Achem|female|14.0|
+----------+-----+--------------------+------+----+



## Problem 3.

Use the DataFrame created by loading data contained in titanic.txt file. Provide reasonable short names for all columns of that DataFrame. Use DataFrame API and SQL API  to create new DataFrames in which every passenger’s ticket costs 20% less than the originally paid prices, and the ordinal number is multiplied by 10 and provided as a new column with name new_number. Again using both DataFrame API and SQL API select gender, name and age of passengers older than 35 years. Display the content of that DataFrame.

In [16]:
# converted titanic passengers data Dataframe API

conv_passengers_df = passengersDF.select(
    passengersDF[0], 
    expr("price - (price * .20) as less_price"),
    passengersDF[2], 
    passengersDF[3], 
    passengersDF[4],
    expr("ord_number * 10 as new_number")
)

conv_passengers_df.show()

+----------+----------+--------------------+------+---+----------+
|ord_number|less_price|                name|gender|age|new_number|
+----------+----------+--------------------+------+---+----------+
|         1|     100.0|     Mr. Owen Harris|  male| 22|      10.0|
|         2|      60.0|Mrs. Florence Bri...|female| 38|      20.0|
|         3|      60.0|Madame. Laina Hei...|female| 66|      30.0|
|         4|      60.0|Jacques Heath (Li...|female| 35|      40.0|
|         5|     100.0|       William Henry|  male| 35|      50.0|
|         6|     100.0|         James Moran|  male|  0|      60.0|
|         7|     140.0|  Timothy J McCarthy|  male| 55|      70.0|
|         8|      20.0|Master Gosta Leon...|  male|  2|      80.0|
|         9|      60.0|Mrs. Oscar Johnso...|female| 27|      90.0|
|        10|      40.0|   Miss. Adele Achem|female| 14|     100.0|
+----------+----------+--------------------+------+---+----------+



In [17]:
#  converted titanic passengers data SQL API

passengersDF.createGlobalTempView("passengers")

sql_pass_df = spark.sql("select ord_number, price - (price * .20) as less_price, name, gender, age, ord_number * 10 as new_number from global_temp.passengers")

In [18]:
# Show data

sql_pass_df.show()

+----------+----------+--------------------+------+---+----------+
|ord_number|less_price|                name|gender|age|new_number|
+----------+----------+--------------------+------+---+----------+
|         1|     100.0|     Mr. Owen Harris|  male| 22|      10.0|
|         2|      60.0|Mrs. Florence Bri...|female| 38|      20.0|
|         3|      60.0|Madame. Laina Hei...|female| 66|      30.0|
|         4|      60.0|Jacques Heath (Li...|female| 35|      40.0|
|         5|     100.0|       William Henry|  male| 35|      50.0|
|         6|     100.0|         James Moran|  male|  0|      60.0|
|         7|     140.0|  Timothy J McCarthy|  male| 55|      70.0|
|         8|      20.0|Master Gosta Leon...|  male|  2|      80.0|
|         9|      60.0|Mrs. Oscar Johnso...|female| 27|      90.0|
|        10|      40.0|   Miss. Adele Achem|female| 14|     100.0|
+----------+----------+--------------------+------+---+----------+



In [19]:
# Passengers older than 35

passengersDF.filter(passengersDF['age'] < 35).select(passengersDF['name'], passengersDF['age']).show()

+--------------------+---+
|                name|age|
+--------------------+---+
|     Mr. Owen Harris| 22|
|         James Moran|  0|
|Master Gosta Leon...|  2|
|Mrs. Oscar Johnso...| 27|
|   Miss. Adele Achem| 14|
+--------------------+---+



In [ ]:
# Passengers > 35 years of age SQL API

sql_age_df = spark.sql("select name, age from global_temp.passengers where age < 35")

In [ ]:
#show data

sql_age_df.show()

+--------------------+----+
|                name| age|
+--------------------+----+
|     Mr. Owen Harris|22.0|
|         James Moran| 0.0|
|Master Gosta Leon...| 2.0|
|Mrs. Oscar Johnso...|27.0|
|   Miss. Adele Achem|14.0|
+--------------------+----+



## Problem 4.

Read file books.json as a DataFrame and than persist that DataFrame as a CSV file and a Parquet file. Subsequently, read the data from those files into new DataFrames. Demonstrate that the data content is not changed.

In [ ]:
# Read Books json with schema into a dataframe

books_df = spark.read.option("multiline", "true").schema(book_schema).json("books.json")

books_df.printSchema()

books_df.show()

root
 |-- title: string (nullable = true)
 |-- author: string (nullable = true)
 |-- published: date (nullable = true)
 |-- publisher: string (nullable = true)
 |-- website: string (nullable = true)

+--------------------+----------------+----------+---------------+--------------------+
|               title|          author| published|      publisher|             website|
+--------------------+----------------+----------+---------------+--------------------+
|Eloquent JavaScri...|Marijn Haverbeke|2014-12-14|No Starch Press|http://eloquentja...|
|Learning JavaScri...|     Addy Osmani|2012-07-01| O'Reilly Media|http://www.addyos...|
+--------------------+----------------+----------+---------------+--------------------+



In [ ]:
# Write to csv file

books_df.write.option("header",True).option("infer_schema", True).csv("/home/manny/dev/cscie-63/hw02/output")

In [ ]:
# Write to parquet

books_df.write.option("header",True).option("infer_schema", True).parquet("/home/manny/dev/cscie-63/hw02/outputII")

In [ ]:
# Read output csv file to dataframe

books_csv_df = (
    spark.read.format("csv")
    .option("header",True)
    .load("/home/manny/dev/cscie-63/hw02/output/part-00000-00b8b13a-3a33-4866-aa49-4538400795d4-c000.csv")
)

books_csv_df.printSchema()

books_csv_df.show()

root
 |-- title: string (nullable = true)
 |-- author: string (nullable = true)
 |-- published: string (nullable = true)
 |-- publisher: string (nullable = true)
 |-- website: string (nullable = true)

+--------------------+----------------+----------+---------------+--------------------+
|               title|          author| published|      publisher|             website|
+--------------------+----------------+----------+---------------+--------------------+
|Eloquent JavaScri...|Marijn Haverbeke|2014-12-14|No Starch Press|http://eloquentja...|
|Learning JavaScri...|     Addy Osmani|2012-07-01| O'Reilly Media|http://www.addyos...|
+--------------------+----------------+----------+---------------+--------------------+



In [ ]:
# Read from parquet output

books_parq_df = (
    spark.read.format("parquet")
    .option("header",True)
    .load("/home/manny/dev/cscie-63/hw02/outputII/part-00000-b8f24d25-d063-4fb3-adea-c701a6d88147-c000.snappy.parquet")
)

books_parq_df.printSchema()

books_parq_df.show()

root
 |-- title: string (nullable = true)
 |-- author: string (nullable = true)
 |-- published: date (nullable = true)
 |-- publisher: string (nullable = true)
 |-- website: string (nullable = true)

+--------------------+----------------+----------+---------------+--------------------+
|               title|          author| published|      publisher|             website|
+--------------------+----------------+----------+---------------+--------------------+
|Eloquent JavaScri...|Marijn Haverbeke|2014-12-14|No Starch Press|http://eloquentja...|
|Learning JavaScri...|     Addy Osmani|2012-07-01| O'Reilly Media|http://www.addyos...|
+--------------------+----------------+----------+---------------+--------------------+

